In [34]:
import json

import whisper

In [2]:
model = whisper.load_model("base")

100%|███████████████████████████████████████| 139M/139M [00:04<00:00, 30.3MiB/s]


In [3]:
result = model.transcribe("short.mp3", verbose=True)

/Users/bwen/personal/chatVideo/.venv/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: English
[00:00.000 --> 00:08.160]  Can I ask just before we get started does IBM own WebX or something?
[00:08.160 --> 00:10.800]  I hate WebX.
[00:10.800 --> 00:11.800]  No offense.
[00:11.800 --> 00:12.800]  Why are you going to go there?
[00:12.800 --> 00:13.960]  Why are you going to go there?
[00:13.960 --> 00:16.320]  No, I don't think Cisco owns WebX.
[00:16.320 --> 00:17.320]  Okay.
[00:17.320 --> 00:18.400]  Yeah, you're right.
[00:18.400 --> 00:23.800]  It always makes me install some software.
[00:23.800 --> 00:29.240]  I think all of the trick up there, UI stuff right underneath that thing in the non colored
[00:29.240 --> 00:30.240]  action box.
[00:30.240 --> 00:34.840]  There's usually a thing that says skip and join his guest or the other part too.
[00:34.840 --> 00:35.840]  I'm in the same boat.
[00:35.840 --> 00:39.720]  Once you've clicked it and you've bo

In [5]:
result.keys()

dict_keys(['text', 'segments', 'language'])

In [35]:
segment = json.load(open("2.json", "r"))

In [11]:
segment = result['segments']

output = [f"{entry['start']}-{entry['end']}:{entry['text']}" for entry in segment]


In [13]:
import re

In [37]:
with open("2.txt", "r") as f:
    lines = f.readlines()

In [18]:
def convert_to_seconds(time_str):
    h, m, s = time_str.split(":")
    return int(h) * 3600 + int(m) * 60 + float(s)

In [38]:
# Parse speaker label file
speaker_segments = []
for line in lines:
    match = re.match(r"(.*?)\s(\d{2}:\d{2}:\d{2}\.\d{3}) --> (\d{2}:\d{2}:\d{2}\.\d{3})", line)
    if match:
        speaker = match.group(1)
        start = match.group(2)
        end = match.group(3)
        # Convert timestamp to seconds
        start_seconds = convert_to_seconds(start)
        end_seconds = convert_to_seconds(end)
        speaker_segments.append((start_seconds, end_seconds, speaker))

In [39]:
# Assuming 'result' is your dictionary
# segment = result['segments']

# Add speaker name to segments
i = 0  # Index for 'segment'
j = 0  # Index for 'speaker_segments'

while i < len(segment) and j < len(speaker_segments):
    start = segment[i]['start']
    end = segment[i]['end']
    speaker_start, speaker_end, speaker = speaker_segments[j]

    # Calculate dynamic tolerance as half the minimum gap to the next or previous end time
    next_end = segment[i+1]['end'] if i+1 < len(segment) else float('inf')
    prev_end = segment[i-1]['end'] if i-1 >= 0 else float('-inf')
    tolerance = min(next_end - end, end - prev_end) / 2.0
    segment[i]['speaker'] = speaker

    if abs(end - speaker_end) <= tolerance:  # Case 1: Entries match
        i += 1
        j += 1
    elif end > speaker_end:  # Case 2: One entry in 'segment' matches multiple in 'speaker_segments'
        j += 1
    else:  # Case 3: Multiple entries in 'segment' match one in 'speaker_segments'
        i += 1

In [24]:
speaker_segments

[(1.42, 9.7, 'Doug (External)'),
 (9.78, 14.82, 'JEFF ROGERS (IBM)'),
 (14.86, 21.14, 'Doug (External)'),
 (21.22, 23.14, 'JEFF ROGERS (IBM)'),
 (24.74, 39.9, 'Matt McDonald (External)'),
 (40.42, 42.34, 'Doug (External)'),
 (42.34, 54.5, 'Matt McDonald (External)'),
 (56.42, 59.62, 'BO WEN (IBM)')]

In [40]:
output = [f"{entry['speaker']}@{entry['start']}-{entry['end']}:{entry['text']}" for entry in segment]
